In [1]:
import torch
from models import TrajectoryGenerator, TrajectoryDiscriminator
from train import init_weights, get_dtypes

In [2]:
long_dtype, float_dtype = get_dtypes()

In [3]:
generator = TrajectoryGenerator()
generator.apply(init_weights)
generator.type(float_dtype).train()
print('Here is the generator:')
print(generator)

Here is the generator:
TrajectoryGenerator(
  (encoder): Encoder(
    (encoder): LSTM(16, 64)
    (spatial_embedding): Linear(in_features=2, out_features=16, bias=True)
  )
  (sattn): SocialAttention(
    (spatial_embedding): Linear(in_features=2, out_features=16, bias=True)
    (mlp_pre_attn): Sequential(
      (0): Linear(in_features=80, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=32, bias=True)
      (3): ReLU()
    )
    (attn): Linear(in_features=2048, out_features=64, bias=True)
  )
  (pattn): PhysicalAttention(
    (spatial_embedding): Linear(in_features=2, out_features=16, bias=True)
    (pre_att_proj): Linear(in_features=512, out_features=16, bias=True)
    (mlp_pre_attn): Sequential(
      (0): Linear(in_features=32, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=32, bias=True)
      (3): ReLU()
    )
    (attn): Linear(in_features=28800, out_features=900, bias=True)
  )
  (deco

In [4]:
discriminator = TrajectoryDiscriminator()
discriminator.apply(init_weights)
discriminator.type(float_dtype).train()
print('Here is the discriminator:')
print(discriminator)

Here is the discriminator:
TrajectoryDiscriminator(
  (encoder): Encoder(
    (encoder): LSTM(16, 64)
    (spatial_embedding): Linear(in_features=2, out_features=16, bias=True)
  )
  (real_classifier): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
    (3): ReLU()
  )
)


In [6]:
vgg = torch.randn((1,512,84,75)).to("cuda")
# seqlen - numpeds - maxpeds - points
x = torch.randn((8,7,64,2)).to("cuda")
x_rel = torch.randn((8,7,64,2)).to("cuda")
# numpeds - x - y
output = generator(x,x_rel,vgg)
print(">> ", output.shape)

>>  torch.Size([12, 7, 2])


In [10]:
print(output[:,2,:])

tensor([[-0.2598, -0.1227],
        [-0.2844, -0.0958],
        [-0.2840, -0.0213],
        [-0.2723,  0.0331],
        [-0.2599,  0.0699],
        [-0.2505,  0.0940],
        [-0.2441,  0.1096],
        [-0.2402,  0.1198],
        [-0.2379,  0.1265],
        [-0.2366,  0.1309],
        [-0.2358,  0.1339],
        [-0.2353,  0.1359]], device='cuda:0', grad_fn=<SliceBackward0>)


In [13]:
output_dis = discriminator(1,output)
print(output_dis.shape)
print(output_dis)

torch.Size([7, 1])
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], device='cuda:0', grad_fn=<ReluBackward0>)
